In [49]:
import pandas
import datetime

site_id = 6

weather = pandas.read_csv(f"site_{site_id}_weather.csv", sep=";", parse_dates=['Timestamp'])
consumption = pandas.read_csv(f"site_{site_id}_consumption.csv", sep=";", parse_dates=['Timestamp'])

def datetime_to_epoch(d1):
    """
    January 1st, 1970 at 00:00:00 UTC is referred to as the Unix epoch
    :param d1: input date
    :return: seconds since unix epoch
    """
    if not d1.tzinfo:
        raise ValueError("date is missing timezone information")

    d2 = datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc)
    time_delta = d1 - d2
    ts = int(time_delta.total_seconds())
    return ts

def to_unix_epoch(row):
    return datetime_to_epoch(row["Timestamp"])

weather = weather[weather["Distance"] < 8]

weather["UnixTS"] = weather.apply(to_unix_epoch, axis=1)
consumption["UnixTS"] = consumption.apply(to_unix_epoch, axis=1)

def convert_to_celisus(f):
    return (f - 32) / 1.8

def find_closest_temperature_at_ts(row):
    timestamp = row["UnixTS"]
    loc = weather["UnixTS"].searchsorted(timestamp)
    
    #return weather.iloc[loc]["Temperature"]

    if loc < len(weather)-1:
        c0 = weather.iloc[ loc ]
        c1 = weather.iloc[ loc+1 ]
        distance = c1["UnixTS"] - c0["UnixTS"]
        alpha = (timestamp - c0["UnixTS"]) / distance
        t0 = weather.iloc[loc]["Temperature"]
        t1 = weather.iloc[loc+1]["Temperature"]
        return (1-alpha) * t0 + alpha * t1
    return weather.iloc[loc]["Temperature"]

consumption["Temperature"] = consumption.apply(find_closest_temperature_at_ts, axis=1)

weather.to_csv(f"site_{site_id}_weather_post.csv", sep=";", index=False)
consumption.to_csv(f"site_{site_id}_consumption_post.csv", sep=";", index=False)

